<a href="https://colab.research.google.com/github/uanak/DXbootcamp/blob/main/bootCamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

import MeCab
import pytorch


In [29]:
mecab = MeCab.Tagger("-Ochasen")
# おまじない
mecab.parse('')

def bunkai(contents):
    ret = []
    for  content in contents:
        ret.append(bunkaiChi(content))
    return ret

def bunkaiChi(content):
  return [token for token in doMecab(content)]

def doMecab(text):
  # ここでmecabのノードクラスに分解して格納
  nodes = mecab.parseToNode(text)
  while nodes:
    if nodes.feature.split(',')[0] == '名詞':
      yield nodes.surface.lower()
    nodes = nodes.next


test1 = ["私の好きな天気は雨です","意味がわからないこともだんだんわかると嬉しい"]
wordlist = bunkai(test1)
print(wordlist)

[['私', '好き', '天気', '雨'], ['意味', 'こと']]


In [ ]:
test = ["a","b"]
while test:
  print (test)
  test = test.next

In [18]:
def test(aaa):
  aaaa = mecab.parse(aaa)
  return aaaa

aa =test("私の好きなお酒はラム酒です")

print(aa)

私	ワタシ	私	名詞-代名詞-一般		
の	ノ	の	助詞-連体化		
好き	スキ	好き	名詞-形容動詞語幹		
な	ナ	だ	助動詞	特殊・ダ	体言接続
お	オ	お	接頭詞-名詞接続		
酒	サケ	酒	名詞-一般		
は	ハ	は	助詞-係助詞		
ラム	ラム	ラム	名詞-一般		
酒	シュ	酒	名詞-接尾-一般		
です	デス	です	助動詞	特殊・デス	基本形
EOS

